In [15]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/MyDrive/robust_DL

ModuleNotFoundError: No module named 'google'

In [16]:
from __future__ import print_function
import os
import argparse
import torch.nn as nn
import torchvision
import numpy as np
import torch.nn.functional as F
from losses.trades import trades_loss
import copy
import torch
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from models.wideresnet import *
from models.resnet import *
from models.small_cnn import *
from torch.utils.data import Dataset, DataLoader
from models.AlexNet import AlexNet
from torch.utils.data.sampler import SubsetRandomSampler

In [17]:
class Data:
  def __init__(self, train_loader, valid_loader, test_loader, attack_loader):
    self.train_loader = train_loader
    self.valid_loader = valid_loader
    self.test_loader = test_loader
    self.attack_loader = attack_loader

class Model:
  model = None
  def __init__(self, id):
    self.id = id

class Loss:
  def __init__(self, loss_fn, id=None):
    self.loss_fn = loss_fn
    self.id = id

class Configuration:
  def __init__(self, data, model, loss, attack, model_pt=None, id=None):
    self.data = data
    self.model = model
    self.loss = loss
    self.attack = attack
    self.model_pt = model_pt # Should move this to model

    self.id = id

  def getConfig(self):
    return self.data, self.model, self.loss, self.attack

  def getId(self):
    return self.id

class CIFAR10CDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

def general_trades_loss_fn(beta=6.0, epsilon=0.3, step_size=0.007, num_steps=10):
  def trades_loss_fn(model, data, target, optimizer):
    return trades_loss(model=model, x_natural=data, y=target, optimizer=optimizer, step_size=step_size,
                      epsilon=epsilon, perturb_steps=num_steps, beta=beta, distance='l_inf')
  return trades_loss_fn

def identity_attack(model, X, y):
  out = model(X)
  acc = (out.data.max(1)[1] == y.data).float().sum()
  return acc.item()



In [ ]:

def accuracy(model, data_loader, device):
    print('claculating accuracy')
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader):
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    return 100. * correct / total

def robust_accuracy(model, attack, data_loader, device):
    print('evaluating robust accuracy')
    model.eval()
    correct = 0
    total = 0

    for data, target in data_loader:
        data, target = data.to(device), target.to(device)

        X, y = Variable(data, requires_grad=True), Variable(target)
        correct_count = attack(model, X, y)
        correct += correct_count
        total += target.size(0)
    return 100. * correct / total 

def train(model, data_loader, valid_dataloader, loss, config, epochs, eval_interval, device):
  print('starting train')

  optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
  # TODO: We can move the optimizer to a field of Loss object

  model.to(device)
  if config.model_pt is not None:
    model.load_state_dict(torch.load(config.model_pt))

  best_eval_acc = 0.0
  patience = 5  # number of VAL Acc values observed after best value to stop training

  for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    for batch_idx, (data, target) in enumerate(data_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        l = loss.loss_fn(model, data, target, optimizer)
        l.backward()
        optimizer.step()
        total_loss += l.item()

        print(loss.id + " @ Batch idx " + str(batch_idx) + " / " + str(len(data_loader) - 1))
    if epoch ==1 or epoch % eval_interval == 0 or epoch == epochs:
      eval_acc= accuracy(model, valid_dataloader, device)

      if (eval_acc > best_eval_acc):  # best so far so save checkpoint to restore later
        best_eval_acc = eval_acc
        patience_count = 0
        torch.save(model.state_dict(), os.path.join("weights", loss.id + ".pt"))
        torch.save(optimizer.state_dict(), os.path.join("optimizers", loss.id +  ".tar"))
      else:
          patience_count += 1
          if patience_count >= patience:
              print(f"Early Stopping!, epoch {epoch}")
              break

    

  return total_loss


In [18]:
def run_experiment(num_epoch=2, valid_size=0.2, eval_interval=1):
  transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),])
  transform_test = transforms.Compose([
    transforms.ToTensor(),])
  use_cuda = torch.cuda.is_available()
  kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
  transform_attack = transforms.Compose([transforms.ToTensor(),])

  trainset = torchvision.datasets.CIFAR10(root='data', train=True, download=True, transform=transform_train)
  validset = torchvision.datasets.CIFAR10(root='data', train=True, download=True, transform=transform_train)
  testset = torchvision.datasets.CIFAR10(root='data', train=False, download=True, transform=transform_test)
  num_train = len(trainset)
  indices = list(range(num_train))
  split = int(np.floor(valid_size * num_train))
  train_idx, valid_idx = indices[split:], indices[:split]
  train_sampler = SubsetRandomSampler(train_idx)
  valid_sampler = SubsetRandomSampler(valid_idx)


  cifar10_train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, sampler=train_sampler, **kwargs)
  cifar10_valid_loader = torch.utils.data.DataLoader(trainset , batch_size=128, sampler=valid_sampler, **kwargs)
  cifar10_test_loader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, **kwargs)


  transform_cifar10c = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])
  images = np.load('data/CIFAR-10-C/spatter.npy') # Set this to whatever we want
  labels = np.load('data/CIFAR-10-C/labels.npy')
  cifar10c_dataset = CIFAR10CDataset(data=images[:200],labels=labels[:200],transform=transform_cifar10c)
  cifar10c_attack_loader = DataLoader(cifar10c_dataset, batch_size=200, shuffle=False)

  cifar10_c_data = Data(cifar10_train_loader, cifar10_valid_loader,cifar10_test_loader, cifar10c_attack_loader)
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  for beta in [0.6]:
    id = f'CIFARC10:Alexnet:TRADES_LOSS:BETA={beta}'
    # model_pt = os.path.join("weights", f'CIFARC10:RESNET18:TRADES_LOSS:BETA={beta}_ep=0.pt')
    model_pt=None
    # resnet18 = ResNet18().to(device)
    alexnet = AlexNet().to(device)
    trades_loss_beta = Loss(general_trades_loss_fn(beta=beta), id)
    config1 = Configuration(cifar10_c_data, alexnet, trades_loss_beta, identity_attack, model_pt, trades_loss_beta.id)
    configurations = []
    configurations.append(config1)

    training_loss = {}
    natural_accuracy = {}
    robustness_accuracy = {}

    for (c, configuration) in enumerate(configurations):
       data, model, loss, attack = configuration.getConfig()
       training_loss[configuration.getId()] = train(model, data.train_loader,data.valid_loader,  loss, configuration,num_epoch,eval_interval, device)
       print(training_loss[configuration.getId()])
       natural_accuracy[configuration.getId()] = accuracy(model, data.test_loader, device)
       print(natural_accuracy[configuration.getId()])
       robustness_accuracy[configuration.getId()] = robust_accuracy(model, attack, data.attack_loader, device)
       print(robustness_accuracy[configuration.getId()] )

In [ ]:
run_experiment(num_epoch=70, valid_size=0.2, eval_interval=1)